 # 🤖 Agente LangChain - Notícias do Botafogo



 Este notebook implementa um agente que coleta notícias do Botafogo e as processa usando LLM no OCI.



 ## Funcionalidades:

 - 📰 Coleta notícias do GE Globo

 - 🤖 Processa com LLM via OCI

 - 🔍 Busca inteligente com LangChain

 ## 🕷️ Coletor de Notícias do Botafogo

In [6]:
!pip install tavily-python
!pip install -U langchain-community langchain-core oci



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 4.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.72
    Uninstalling langchain-core-0.3.72:
      Successfully uninstalled langchain-core-0.3.72


In [7]:
import requests
from langchain_core.tools import tool
from typing import Dict, List
import re
from tavily import TavilyClient

@tool
def get_botafogo_comprehensive_news(pergunta:str="Qual proximo jogo do Botafogo?") -> str:
    """Retorna notícias do Botafogo de múltiplas fontes do GE Globo"""
    
    client = TavilyClient("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") #essa é a API de dev, pode ser utilizada de qualquer forma
    response = client.search(
    query=pergunta)
    return response



 ## 🤖 Configuração de autenticação de OCI SDK
    - https://github.com/oracle/oci-python-sdk/blob/master/examples/configuration_example.py

 ## 🤖 Criação do Agente LangChain

In [8]:

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOCIGenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

# 2. Configuração da LLM OCI
llm = ChatOCIGenAI(
  model_id="",
  service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
  compartment_id="",
  provider="meta",
  model_kwargs={
    "temperature": 0.3, 
    "max_tokens": 800,   
    "top_p": 0.8,  
    "frequency_penalty": 0,
    "presence_penalty": 0,
  },
  auth_type="API_KEY",
  auth_profile="DEFAULT"
)
tools = [
    get_botafogo_comprehensive_news
]

react_prompt_template = """You are a helpful assistant specialized in Brazilian football teams news and information. 
Answer the following questions as best you can using the available tools.

IMPORTANT: After using any tool, you MUST analyze and summarize the results in your own words. 
Do not just copy the raw output - provide a meaningful summary and insights. Do not use old news.

You have access to the following tools:

{tools}

Use the following format EXACTLY:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action (use empty string "" if no input needed)
Observation: the result of the action
Thought: Now I need to analyze this information and provide a helpful summary
Final Answer: A comprehensive answer based on the tool results, with analysis and insights in Brazilian Portuguese

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(react_prompt_template)

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    max_iterations=3  # Limita o número de iterações para evitar loops infinitos
)
pergunta = "Qual próximo jogo do Botafogo?"

resposta = agent_executor.invoke({"input": pergunta})




> Entering new AgentExecutor chain...
Para responder a essa pergunta, preciso buscar informações atualizadas sobre a agenda do Botafogo. 
A melhor forma de fazer isso é utilizando a função get_botafogo_comprehensive_news, que fornece notícias do Botafogo de múltiplas fontes do GE Globo.

Action: get_botafogo_comprehensive_news
Action Input: Qual próximo jogo do Botafogo?
{'query': 'Qual próximo jogo do Botafogo?\n', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.365scores.com/pt-br/football/team/botafogo-1211', 'title': 'Botafogo: Jogos e Resultados ao vivo - 365Scores', 'content': '... Botafogo. Botafogo fará o seu próximo jogo contra Vasco da Gama em quinta-feira, 28 de agosto de 2025 (Vasco da Gama x Botafogo). Botafogo fez sua última', 'score': 0.9154545, 'raw_content': None}, {'url': 'https://www.placardefutebol.com.br/time/botafogo/proximos-jogos', 'title': 'Próximos jogos do Botafogo - calendário, datas e horários', 'content': 'Próx

In [9]:
from IPython.display import Markdown
analysis_prompt = f"""
Com base nas seguintes informações sobre o Botafogo, forneça um resumo claro e útil:

DADOS COLETADOS:
{resposta}

Por favor, analise essas informações e forneça:
1. Resposta direta à pergunta: "{pergunta}"

Responda de forma conversacional e útil, não apenas liste os dados.
"""

analysis_response = llm.invoke([HumanMessage(content=analysis_prompt)])
display(Markdown(analysis_response.content))

Se você está procurando saber sobre o próximo jogo do Botafogo, posso ajudar! De acordo com as informações disponíveis, o próximo jogo do Botafogo está agendado para o dia 28 de agosto de 2025, às 21:30, e será contra o Vasco da Gama. Além disso, o time tem outros jogos importantes nos próximos dias, incluindo partidas contra o Bragantino e o São Paulo.

É sempre uma boa ideia verificar as fontes oficiais do clube ou do campeonato para ter as informações mais atualizadas sobre a agenda do time, pois os jogos podem ser adiados ou ter seus horários alterados. Mas, para o que sabemos até agora, o jogo contra o Vasco da Gama é o próximo grande desafio do Botafogo!